In [1]:
import pandas as pd 
import wandb

api = wandb.Api()
entity, project = "<entity>", "<project>"  # set to your entity and project 
runs = api.runs(entity + "/" + project) 

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
         if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

runs_df.to_csv("project.csv")

ValueError: Could not find project <project>

# Extract data from run

In [2]:
import requests
import os
import pathlib
from typing import (
    Any,
    Generator,
    IO,
    Optional,
    Union,
)
def fsync_open(
    path: Union[pathlib.Path, str], mode: str = "w"
) -> Generator[IO[Any], None, None]:
    """
    Opens a path for I/O, guaranteeing that the file is flushed and
    fsynced when the file's context expires.
    """
    with open(path, mode) as f:
        yield f

        f.flush()
        os.fsync(f.fileno())

def download_file_from_url(source_url: str, api_key: Optional[str] = None
) -> None:
    response = requests.get(source_url, auth=("api", api_key), stream=True, timeout=5)
    response.raise_for_status()
    bytes_list = []
    for data in response.iter_content(chunk_size=1024):
        bytes_list.append(data)
    final_byte_data = b''.join(bytes_list)
    return final_byte_data

In [3]:
import wandb
import json
import pandas as pd
api = wandb.Api()
# https://docs.wandb.ai/ref/python/public-api/run#files
run = api.run("dschaub/rltsp/2xs4gzqx")
summary = run.summary._json_dict
config = {k: v for k,v in run.config.items() if not k.startswith('_')}
name = run.name
run.summary
# summary_df = pd.DataFrame(summary)
# # run_df = pd.DataFrame({
# #    "summary": summary,
# #    "config": config,
# #    "name": name
# summary['run_data']
from io import BytesIO

for file in run.files():
   if "table" in str(file):
      # table = json.load(file.download())
      bytes_data = download_file_from_url(file.url, wandb.Api().api_key)
      data_dict = json.loads(bytes_data.decode('utf-8'))
      # csv_data = BytesIO(bytes_data)
      # df = pd.read_csv(BytesIO(bytes_data))
      df = pd.DataFrame(data=data_dict["data"], columns=data_dict["columns"])
      print(df)

     sample_id  approx_errors  pred_lengths
0            0   7.478901e-06      3.846662
1            1   2.190074e+00      3.770207
2            2   1.787784e+00      4.083383
3            3   2.126051e+00      4.521512
4            4   7.136312e-06      3.886870
..         ...            ...           ...
123        123   1.415768e+00      4.177987
124        124   9.192172e-02      4.353903
125        125   7.037997e-01      4.009487
126        126   5.795264e+00      4.065396
127        127  -4.412188e-07      3.442972

[128 rows x 3 columns]
    Instance 0  Instance 1  Instance 2  Instance 3  Instance 4  Instance 5  \
0            0           0           0           0           0           0   
1           19           7           2           1          11          14   
2           13           8           9          15          10           6   
3            9           1          13          12           8           7   
4            6           9           3           8        

In [ ]:
download_file_from_url(wandb.Api().api_key)

# Extract data from sweep

In [ ]:
import wandb
api = wandb.Api()

sweep = api.sweep("<entity>/<project>/<sweep_id>")
sweep_runs = sweep.runs